In [ ]:
from build.passwords import password, jupyterhub_user, user
import requests
from requests.auth import HTTPBasicAuth

api_base_url = "https://engr-131-api.eastus.cloudapp.azure.com/"

In [ ]:
params = {"username": "jca92"}
res = requests.get(
    url=api_base_url.rstrip("/") + "/student-grades-testing",
    params=params,
    auth=HTTPBasicAuth(user(), password()),
)

In [ ]:
api_base_url.rstrip("/") + "/student-grades-testing"

In [ ]:
[assignments, sub] = res.json()

In [ ]:
from grade_reports.grade_reports import get_student_grades

In [ ]:
assignments, submissions = get_student_grades("jca92")

In [ ]:
submissions

In [ ]:
import pandas as pd

# Define custom sort order for 'Category'
custom_order = ["lecture", "homework", "lab", "attendance", "readings", "quiz", "practice quiz" ]
assignments["assignment_name"] = pd.Categorical(assignments["assignment_name"], categories=custom_order, ordered=True)

# Sort by 'Category' (custom order) and then by 'Value'
sorted_df = assignments.sort_values(by=["assignment_name", "week_number"])


# Get unique pairs of 'week_number' and 'assignment_name'
unique_pairs = sorted_df[["week_number", "assignment_name"]].drop_duplicates()

# Build an iterator over the unique pairs
pair_iterator = unique_pairs.itertuples(index=False, name=None)

In [ ]:
# Filter rows matching each pair
for week_number, assignment_name in pair_iterator:
    matching_rows = sorted_df[
        (sorted_df["week_number"] == week_number)
        & (sorted_df["assignment_name"] == assignment_name)
    ]
    
    # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
    title = matching_rows["title"]
    
    # extracts the names of the assignments grades which are associated with a given assignment, this helps if two assignments have the same name.
    for assignment_db_name in matching_rows["title"]:
        print(assignment_db_name)
        # now we need the logic to get the best max score based on the assignment name and submission time. 
        
        submissions[submissions["assignment"]==assignment_db_name]
    

In [ ]:
sub